In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

c:\Users\승범 pc\Desktop\공부\딥러닝\엘지에이머스


In [72]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
from sklearn.preprocessing import StandardScaler
seed_everything()  # NOTE: default 42
_model = Model()

In [73]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [165]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory

# Main

In [166]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["apply"], "params": pkls.get("product_category")},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
    ]
}

## preprocessing

In [172]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

## encoding

In [173]:
from sklearn.preprocessing import LabelEncoder
class Encode:
    def __init__(self):
        pass
    def label_encoder(self,df,columns):
        if not encoding_zip['label']:
            for col in columns: # train
                encoder = LabelEncoder()
                try:
                    df.loc[:,col] = encoder.fit_transform(df[col])
                except Exception as e:
                    print(e)
                    print(col)
                # 인코더 저장
                encoding_zip['label'].append(encoder)
        else: # test
            for idx, col in enumerate(columns):
                encoder = encoding_zip['label'][idx]
                
                for label in np.sort(df[col].unique()):
                    if label not in encoder.classes_:
                        encoder.classes_ = np.append(encoder.classes_,label)
                df.loc[:,col] = encoder.transform(df[col])
            
        return df

In [174]:
encode = Encode()
encoding_zip = {"label":[], "onehot":[]}

In [175]:
exisiting_columns = [
                    'business_area','business_subarea','business_unit',
#                         'customer_type','customer_job','customer_position'
#                         'inquiry_type','expected_timeline',
                        'enterprise',
#                         'response_corporate',"customer_country"
#                         'product_subcategory','product_category',
] 
preprocess_columns = [
                    'category_1','category_2', 'category_3', # product_category
                    'new_inquiry_type','new_expected_timeline', # inquiry
                    'country','continent', # country
                    'customer_type2','job_function','seniority_level' # customer
]

In [176]:
label_encode_columns = exisiting_columns + preprocess_columns

In [177]:
# 결측치 채우기
def fillna(df):
    for col in df.columns:
        if df[col].isna().sum():
            if df[col].dtypes =='O':
                df[col].fillna('ETC', inplace = True)
            else:
                df[col].fillna(0,inplace = True)
    return df
df_train = fillna(df_train)
df_test = fillna(df_test)

In [135]:
# 스캐일러 사용.
#scale = StandardScaler()
#df_train['lead_desc_length'] = scale.fit_transform(df_train['lead_desc_length'].values.reshape(-1, 1))
#df_test['lead_desc_length'] = scale.transform(df_test['lead_desc_length'].values.reshape(-1, 1))

In [178]:
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [179]:
drop_col = ['id_strategic_ver','it_strategic_ver',
            'customer_country.1','response_corporate',"customer_country", # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
            'cate_is_nan','category_3' # product_category
#            'historical_existing_cnt',
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

In [180]:
# train
customer_idx_counts = df_train['customer_idx'].value_counts()
bins = [0, 2, 10, 100, 1000, 10000]
df_counts = pd.cut(customer_idx_counts, bins=bins, labels=False, right=False)
df_train['customer_idx'] = df_train['customer_idx'].map(df_counts)
# test
customer_idx_counts = df_test['customer_idx'].value_counts()
bins = [0, 2, 10, 100, 1000, 10000]
df_counts = pd.cut(customer_idx_counts, bins=bins, labels=False, right=False)
df_test['customer_idx'] = df_test['customer_idx'].map(df_counts)

In [155]:
# 중복제거
#print(len(df_train))
#df_train = df_train.drop_duplicates()
#print(len(df_train))


55792
51584


## train

In [156]:
model = _model.decision_tree

In [183]:
df_train.columns

Index(['bant_submit', 'business_unit', 'com_reg_ver_win_rate', 'customer_idx',
       'enterprise', 'historical_existing_cnt', 'idit_strategic_ver',
       'lead_desc_length', 'ver_cus', 'ver_pro', 'ver_win_rate_x',
       'ver_win_ratio_per_bu', 'business_area', 'business_subarea',
       'lead_owner', 'is_converted', 'country', 'continent',
       'new_inquiry_type', 'new_expected_timeline', 'customer_interest',
       'category_1', 'category_2', 'customer_type2', 'job_function',
       'seniority_level'],
      dtype='object')

In [181]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [182]:
X.reset_index(drop = True, inplace = True)
Y.reset_index(drop = True, inplace = True)

In [184]:
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = defaultdict(list)
train_result = defaultdict(list)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:01,  4.01it/s]

----[K-Fold Train Score]-----
f1 score : 0.9881 / STD: (+/- 0.0002)
precision score : 0.9806 / STD: (+/- 0.0011)
recall score : 0.9957 / STD: (+/- 0.0013)
roc_auc_score score : 0.9970 / STD: (+/- 0.0006)
----[K-Fold Validation Score]-----
f1 score : 0.7845 / STD: (+/- 0.0048)
precision score : 0.7903 / STD: (+/- 0.0126)
recall score : 0.7791 / STD: (+/- 0.0107)
roc_auc_score score : 0.8801 / STD: (+/- 0.0050)


## test & submission

In [191]:
model.feature_importances_

array([0.01179212, 0.0078893 , 0.04298346, 0.10441303, 0.01766558,
       0.42473824, 0.00104566, 0.06843767, 0.00792245, 0.00120848,
       0.00524577, 0.0165652 , 0.0091973 , 0.01494967, 0.11924774,
       0.02303771, 0.01139486, 0.00258305, 0.01304036, 0.00854027,
       0.00591273, 0.0107316 , 0.02049794, 0.030273  , 0.02068682])

In [185]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [186]:
pred = model.predict(test_drop)
pred = np.where(pred == 0, False, True)

In [187]:
df_test['is_converted'] = pred
df_test['is_converted'].value_counts()

False    4125
True     1146
Name: is_converted, dtype: int64

In [188]:
df_test.to_csv("submission.csv",index = False)